In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import numpy as np
import scipy as sp
from scipy.optimize import check_grad
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker
import pandas as pd
from os import path
import seaborn as sns
from matplotlib import gridspec as gs
from sklearn.utils.extmath import cartesian

from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator

from compton import create_observable_set, pol_vec_trans_tex_names
from compton import proton_pol_vec_mean, proton_pol_vec_std
from compton import neutron_pol_vec_mean, neutron_pol_vec_std
from compton import proton_pol_vec_trans_std, proton_pol_vec_trans_mean
from compton import neutron_pol_vec_trans_std, neutron_pol_vec_trans_mean

from compton import plot_subsets, compute_all_1pt_utilities, compute_max_utilities
from compton import plot_comparison_subsets_for_observables
from compton import offset_scatterplot_data
from compton import plot_utilities_all_observables
from compton import plot_comparison_subsets_and_truncation
from compton import convert_max_utilities_to_dataframe, convert_max_utilities_to_flat_dataframe
from compton import ComptonExperiment, create_experiment_infos
from compton import setup_rc_params, light_mode, dark_mode
from compton import observables_name_map, dsg_label
from compton import DesignLabels
from compton import plot_fvr_with_offset

In [2]:
obs_file = path.abspath('../data/polarisabilities-coefficient-table-for-all-observables_20191111_jam.csv')
df = pd.read_csv(obs_file, dtype={'observable': str})
# df['observable'] = df['observable'].replace(observables_name_map)
df['nucleon'] = df['nucleon'].replace({'proton': DesignLabels.proton, 'neutron': DesignLabels.neutron})
observables_unique = df['observable'].unique()

In [3]:
df_fixed = pd.read_csv('../data/compton_observables.csv', index_col=False, dtype={'observable': str})
df_fixed_new = df_fixed.copy()
# df_fixed['observable'] = df_fixed['observable'].replace(observables_name_map)
# df_fixed['nucleon'] = df_fixed['nucleon'].replace({'proton': DesignLabels.proton, 'neutron': DesignLabels.neutron})
# observables_unique = df_fixed['observable'].unique()

In [4]:
df_fixed['observable'].unique()

array(['Y', '2X', '3Y', '1X', '3Yp', '1Xp', 'crosssection', '2Z', '2Xp',
       '1Zp', '1Z', '3', '2Zp'], dtype=object)

In [5]:
df_fixed

,omegalab [MeV],thetalab [deg],y0,y2,y3,y4,observable,nucleon
0,5.0,1,0.0,0.000000,0.000000,0.000000,Y,proton
1,5.0,5,0.0,0.000000,0.000000,0.000000,Y,proton
2,5.0,10,0.0,0.000000,0.000000,0.000000,Y,proton
3,5.0,15,0.0,0.000000,0.000000,0.000000,Y,proton
4,5.0,20,0.0,0.000000,0.000000,0.000000,Y,proton
...,...,...,...,...,...,...,...,...
71183,340.0,160,0.0,-0.045191,0.035490,0.005951,3,neutron
71184,340.0,165,0.0,-0.026357,0.019896,0.003274,3,neutron
71185,340.0,170,0.0,-0.012017,0.008820,0.001432,3,neutron
71186,340.0,175,0.0,-0.003050,0.002201,0.000354,3,neutron


In [6]:
cov_p_proton = np.diag(proton_pol_vec_std)
cov_p_neutron = np.diag(neutron_pol_vec_std)

cov_p_proton_trans = np.diag(proton_pol_vec_trans_std)
cov_p_neutron_trans = np.diag(neutron_pol_vec_trans_std)

In [7]:
compton_obs_no_error = create_observable_set(
    df=df, cov_exp=0., cov_p_proton=cov_p_proton, cov_p_neutron=cov_p_neutron
)

In [8]:
compton_obs_no_error

{('1X', 'Neutron', 3, 'nonlinear'): 1X(3, Neutron),
 ('1X',
  'Neutron',
  3,
  'linear'): 1X(3, Neutron) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1X', 'Neutron', 4, 'nonlinear'): 1X(4, Neutron),
 ('1X',
  'Neutron',
  4,
  'linear'): 1X(4, Neutron) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1X', 'Proton', 3, 'nonlinear'): 1X(3, Proton),
 ('1X',
  'Proton',
  3,
  'linear'): 1X(3, Proton) about [10.65  3.15 -1.1   2.2  -0.4   1.9 ],
 ('1X', 'Proton', 4, 'nonlinear'): 1X(4, Proton),
 ('1X',
  'Proton',
  4,
  'linear'): 1X(4, Proton) about [10.65  3.15 -1.1   2.2  -0.4   1.9 ],
 ('1Xp', 'Neutron', 3, 'nonlinear'): 1Xp(3, Neutron),
 ('1Xp',
  'Neutron',
  3,
  'linear'): 1Xp(3, Neutron) about [11.55        3.65       -3.939477    1.3        -0.0518689   2.18856612],
 ('1Xp', 'Neutron', 4, 'nonlinear'): 1Xp(4, Neutron),
 ('1Xp',
  'Neutron',
  4,
  'linear'): 1Xp(4, Neutron) about [11.55        3.65       

In [9]:
best_pols = {DesignLabels.proton: proton_pol_vec_mean, DesignLabels.neutron: neutron_pol_vec_mean}
for obs in observables_unique:
    for order in [3, 4]:
        for nucleon in [DesignLabels.proton, DesignLabels.neutron]:
            obs2 = obs if obs != 'crosssection' else 'dsg'
            pred = compton_obs_no_error[obs2, nucleon, order, 'nonlinear'](best_pols[nucleon])
            fixed_mask = (df_fixed_new['observable'] == obs) & (df_fixed_new['nucleon'] == nucleon.lower())
            df_fixed_new.loc[fixed_mask, 'y'+str(order)] = pred

In [10]:
for y in ['y0', 'y2', 'y3', 'y4']:
    new_val = df_fixed_new[df_fixed_new['nucleon'] == 'neutron'][y].values
    old_val = df_fixed[df_fixed['nucleon'] == 'neutron'][y].values
    print(np.allclose(new_val, old_val))

True
True
False
False


In [11]:
df_fixed_new.to_csv('../data/compton_observables_new.csv')